In [51]:
import numpy as np
import pandas as pd
import json
import requests
import re
import os
from datetime import datetime as dt
from pandas.io.json import json_normalize
import geocoder
import matplotlib.pylab as plt
import seaborn as sns
import shapefile
%matplotlib inline


### Gathering raw data from data.gov.sg

In [ ]:
url1 = 'https://data.gov.sg/api/action/datastore_search?resource_id=42ff9cfe-abe5-4b54-beda-c88f9bb438ee'
url2 = 'https://data.gov.sg/api/action/datastore_search?resource_id=1b702208-44bf-4829-b620-4615ee19b57c'
url3 = 'https://data.gov.sg/api/action/datastore_search?resource_id=83b2fc37-ce8c-4df4-968b-370fd818138b'
url4 = 'https://data.gov.sg/api/action/datastore_search?resource_id=8c00bf08-9124-479e-aeca-7cc411d884c4'
url5 = 'https://data.gov.sg/api/action/datastore_search?resource_id=adbbddd3-30e2-445f-a123-29bee150a6fe'
urls = [url1, url2, url3, url4, url5]

df_init = pd.DataFrame()
for url in urls:
    result = requests.get(url).json()['result']
    url = url + '&limit={}'.format(result['total'])
    df_init = df_init.append(pd.DataFrame(requests.get(url).json()['result']['records']))

print(len(df_init))

#### Save the loaded data file as csv locally

In [4]:
df_init.to_csv("dat_init.csv", index = False)

#### Check NaN data 

In [136]:
df_init = pd.read_csv("dat_init.csv")

print(len(df_init[df_init['remaining_lease'].isna()]),
len(df_init[df_init['month'].isna()]),
len(df_init[df_init['town'].isna()]),
len(df_init[df_init['flat_type'].isna()]),
len(df_init[df_init['flat_model'].isna()]),
len(df_init[df_init['floor_area_sqm'].isna()]),
len(df_init[df_init['street_name'].isna()]),
len(df_init[df_init['resale_price'].isna()]),
len(df_init[df_init['storey_range'].isna()]),
len(df_init[df_init['block'].isna()]))

709054 0 0 0 0 0 0 0 0 0


#### Get the sparse info remaining lease from commence date

In [138]:
df_init['remaining_lease'] = df_init['lease_commence_date'].astype(int) - 1918

#### Clean the raw data features and features type

In [140]:
# Regular the type of features
df_init['month'] = pd.to_datetime(df_init['month'], format = '%Y-%m')
df_init['year'] = pd.DatetimeIndex(df_init['month']).year
df_init['month'] = pd.DatetimeIndex(df_init['month']).month
df_init[['floor_area_sqm', 'resale_price']] = df_init[['floor_area_sqm', 'resale_price']].astype(float)
df_init[['year', 'lease_commence_date']] = df_init[['year', 'lease_commence_date']].astype(int)


#### Calculate new features 

In [142]:
df_init['price_sqm'] = df_init['resale_price']/df_init['floor_area_sqm']
df_init.head()

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block,year,price_sqm
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,1,61,1979,10 TO 12,1,406,2017,5272.727273
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,1,60,1978,01 TO 03,2,108,2017,3731.343284
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,1,62,1980,01 TO 03,3,602,2017,3910.447761
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,1,62,1980,04 TO 06,4,465,2017,3897.058824
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,1,62,1980,01 TO 03,5,601,2017,3955.223881


#### Clean street_name

In [144]:
df_init['street_name'] = df_init['street_name'].str.replace('UPP ', 'UPPER ', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace(' CTRL', ' CENTRAL', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace('JLN ', 'JALAN ', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace('BT ', 'BUKIT ', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace('LOR ', 'LORONG ', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace(' NTH', ' NORTH', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace('NTH BRIDGE', 'NORTH BRIDGE', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace('KG', 'KAMPONG', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace(' DR', ' DRIVE', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace('TG PAGAR', 'TANJONG PAGAR', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace("C'WEALTH", 'COMMONWEALTH', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace("WOODLANDS DRIVE 75", 'WOODLAND DRIVE 75', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace("TAMAN HO SWEE", 'JALAN BUKIT HO SWEE', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace("YUNG PING RD", '157 CORPORATION DRIVE', regex = True)
df_init['street_name'] = df_init['street_name'].str.replace("YUNG LOH RD", '157 CORPORATION DRIVE', regex = True)

In [145]:
address_set = df_init["street_name"].unique().tolist()
print(len(address_set))

569


### Gathering new data - Coordinate

In [15]:
latitude = np.zeros(len(address_set))
longitude = np.zeros(len(address_set))


for n in range(0,len(address_set)):
    geo = geocoder.osm(address_set[n] + ', SG')
    if geo.x is None:
        latitude[n] = 0
        longitude[n] = 0
    else:
        latitude[n] = geo.y
        longitude[n] = geo.x

latitude, longitude

(array([ 1.3676017 ,  1.3823185 ,  1.3768247 ,  1.3643546 ,  1.3696908 ,
         1.3827747 ,  1.3784539 ,  1.3741564 ,  1.3730573 ,  1.3340399 ,
         1.3297469 ,  1.3313413 ,  1.314061  ,  1.3296754 ,  1.3274218 ,
         1.3339821 ,  1.3198453 ,  1.325289  ,  1.3267815 ,  1.3258683 ,
         1.3569767 ,  1.3247239 ,  1.2846604 ,  1.3313564 ,  1.3376794 ,
         1.326493  ,  1.323155  ,  1.3260581 , -5.406307  ,  1.3313193 ,
         1.3264785 ,  1.3538309 ,  1.3603001 ,  1.3480256 ,  1.3508769 ,
         1.3599872 ,  1.3563979 ,  1.3601554 ,  1.3513259 ,  1.3637268 ,
         1.3556782 ,  1.3499936 ,  1.3543135 ,  1.3468302 ,  1.3548853 ,
         1.354495  ,  1.3515765 ,  1.3559923 ,  1.3496472 ,  1.3598218 ,
         1.3438997 ,  1.3375316 ,  1.3564848 ,  1.274558  ,  1.3426301 ,
         1.287298  ,  1.2728382 ,  1.2891495 ,  1.2774161 ,  1.2734284 ,
         1.2746254 ,  1.2860375 ,  1.2880059 ,  1.2779863 ,  1.28350915,
         1.2900221 ,  1.2859726 ,  1.2741667 ,  1.2

#### Save coordinate data locally for convenience

In [283]:
# df_lat_long = pd.DataFrame(df_init['street_name'].unique())
# df_lat_long['Latitude'] = latitude
# df_lat_long['Longitude'] = longitude
# df_lat_long.to_csv("df_lat_long.csv", index = False)
# df_lat_long.columns = ['street_name','Latitude','Longitude']
# df_lat_long
df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long.columns = ['street_name','Latitude','Longitude']
df_lat_long[df_lat_long['Latitude'] == 0]
df_lat_long[df_lat_long['Longitude'] == 0]
# 1.3201, 103.8873
df_lat_long.loc[df_lat_long['street_name']=='JALAN PASAR BARU','Latitude'] = 1.3201
df_lat_long.loc[df_lat_long['street_name']=='JALAN PASAR BARU','Longitude'] = 103.8873

df_lat_long.loc[df_lat_long['street_name']=='NILE RD','Latitude'] = 1.2828
df_lat_long.loc[df_lat_long['street_name']=='NILE RD','Longitude'] = 103.8175

df_lat_long.loc[df_lat_long['street_name']=='JALAN MEMBINA BARAT','Latitude'] = 1.283347
df_lat_long.loc[df_lat_long['street_name']=='JALAN MEMBINA BARAT','Longitude'] = 103.826121

df_lat_long.loc[df_lat_long['street_name']=='BUANGKOK STH FARMWAY 1','Latitude'] = 1.38838781397
df_lat_long.loc[df_lat_long['street_name']=='BUANGKOK STH FARMWAY 1','Longitude'] = 103.987491735

#### check for none coordinate data

In [286]:
df_lat_long[df_lat_long['Latitude'] == 0]
df_lat_long[df_lat_long['Longitude'] == 0]
df_lat_long.to_csv("df_lat_long.csv")

### Gathering new data - MRT, Bus, Schools, Entertainment, Park
https://data.gov.sg/dataset/sdcp-mrt-station-point?resource_id=1123aaa2-4575-4181-81f6-8e11cda8bd8a%20[D]

In [186]:
df_venues = pd.read_csv("./Data/dat_venues.csv")
df_venues['Venue Category'].unique()

array(['Movie Theater', 'Hotpot Restaurant', 'Chinese Restaurant',
       'Clothing Store', 'Trail', 'Playground', 'Furniture / Home Store',
       'Accessories Store', 'Multiplex', 'Department Store', 'Bookstore',
       'Japanese Restaurant', 'Outlet Store', 'Bakery',
       'Vegetarian / Vegan Restaurant', 'Shoe Store', 'Shopping Mall',
       'Supermarket', 'Bubble Tea Shop', 'Frozen Yogurt Shop',
       'Indian Restaurant', 'Sandwich Place', 'Burger Joint',
       'Coffee Shop', 'Sporting Goods Shop', 'Korean Restaurant',
       'Ramen Restaurant', 'Café', 'Salad Place', 'German Restaurant',
       'Sushi Restaurant', 'Halal Restaurant', 'Juice Bar',
       'Convenience Store', 'Pharmacy', 'Food Court',
       'Gym / Fitness Center', 'Asian Restaurant', 'Market',
       'Fast Food Restaurant', 'Noodle House', 'Stadium', 'Train Station',
       'Food Truck', 'Park', 'Spa', 'Grocery Store', 'Bus Stop',
       'Recreation Center', 'Athletics & Sports', 'Malay Restaurant',
       'Ste

In [179]:
print(len(df_venues[df_venues['Venue Category']=='Train Station']))
print(len(df_venues[df_venues['Venue Category']=='Metro Station']))
print(len(df_venues[df_venues['Venue Category']=='Light Rail Station']))
print(len(df_venues[df_venues['Venue Category']=='Train Station']))
print(len(df_venues[df_venues['Venue Category']=='Mrt']))

23
28
15
23
0


In [187]:
df_venues

,street_name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,JURONG EAST ST 21,1.333969,103.745903,Platinum Movie Suites,1.333425,103.744113,Movie Theater
1,JURONG EAST ST 21,1.333969,103.745903,Hai Di Lao 海底捞火锅 (Hai Di Lao),1.334577,103.746118,Hotpot Restaurant
2,JURONG EAST ST 21,1.333969,103.745903,Song Fa Bak Kut Teh 松發肉骨茶,1.333394,103.743420,Chinese Restaurant
3,JURONG EAST ST 21,1.333969,103.745903,UNIQLO,1.333175,103.743160,Clothing Store
4,JURONG EAST ST 21,1.333969,103.745903,J-Walk,1.333946,103.744433,Trail
...,...,...,...,...,...,...,...
12766,ANG MO KIO ST 61,1.383185,103.840077,Bus Stop 55029 (Seasons Park),1.385679,103.839109,Bus Stop
12767,ANG MO KIO ST 61,1.383185,103.840077,Castle Green BBQ Pit,1.385623,103.841370,BBQ Joint
12768,ANG MO KIO ST 61,1.383185,103.840077,Castle Green Gymnasium,1.385921,103.841753,Gym
12769,ANG MO KIO ST 61,1.383185,103.840077,Gym @ The Calrose,1.382261,103.836902,Gym


#### No enough MRT data, which is essential

In [185]:
# df_mrt = pd.read_csv("https://raw.githubusercontent.com/hxchua/datadoubleconfirm/master/datasets/mrtsg.csv", header=0)
# df_mrt.to_csv("mrt_location.csv")
df_mrt = pd.read_csv("mrt_location.csv")

In [219]:
df_mrt
df_mrt["lat_long"] = df_mrt[['Latitude', 'Longitude']].apply(geopy.Point, axis=1)
mrt_list = list(df_mrt.T.iloc[-1,:])
mrt_list

[Point(1.440585001, 103.8009982, 0.0),
 Point(1.316432612, 103.882893, 0.0),
 Point(1.369933175, 103.8495535, 0.0),
 Point(1.388092704, 103.9054179, 0.0),
 Point(1.380017897, 103.7726667, 0.0),
 Point(1.342828338, 103.8797462, 0.0),
 Point(1.281873788, 103.8590733, 0.0),
 Point(1.281873788, 103.8590733, 0.0),
 Point(1.341223176, 103.77581, 0.0),
 Point(1.323979969, 103.9299587, 0.0),
 Point(1.334742117, 103.9179554, 0.0),
 Point(1.33660783, 103.9322077, 0.0),
 Point(1.29886427, 103.8503799, 0.0),
 Point(1.313672233, 103.8629702, 0.0),
 Point(1.35130868, 103.84915, 0.0),
 Point(1.350838988, 103.8481398, 0.0),
 Point(1.319395706, 103.861679, 0.0),
 Point(1.338604054, 103.7060994, 0.0),
 Point(1.322110193, 103.8149876, 0.0),
 Point(1.322423979, 103.8161362, 0.0),
 Point(1.340471684, 103.8467942, 0.0),
 Point(1.296861687, 103.8506629, 0.0),
 Point(1.382877858, 103.8931042, 0.0),
 Point(1.299550746, 103.8568623, 0.0),
 Point(1.300465076, 103.8557007, 0.0),
 Point(1.349034109, 103.74959, 0.0

In [223]:
# tuple(mrt_list[0])

(1.440585001, 103.8009982, 0.0)

#### Calculate the nearest MRT and distance to Downtown

In [289]:
import geopy
df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long["lat_long"] = df_lat_long[['Latitude', 'Longitude']].apply(geopy.Point, axis=1)

station_distance = pd.DataFrame(df_lat_long["lat_long"])
for x in mrt_list:
    station_distance[tuple(x)]=df_lat_long['lat_long'].apply(lambda y: geopy.distance.geodesic(y,x).km)
# station_distance

station_distance["lat_long"] = station_distance["lat_long"].apply(tuple)
station_distance['distance_to_nearest_mrt'] = station_distance.iloc[:,1:].apply(lambda x: min(x), axis=1)
station_distance = station_distance.iloc[:,[0,-1]]
df_lat_long['distance_to_downtown'] = df_lat_long['lat_long'].apply(lambda y: geopy.distance.geodesic(geopy.Point(1.300567, 103.821658), y).km)
df_lat_long["lat_long"] = df_lat_long["lat_long"].apply(tuple)
df_lat_long = pd.merge(df_lat_long, station_distance, on='lat_long', how='left')
df_lat_long

,Unnamed: 0,street_name,Latitude,Longitude,lat_long,distance_to_downtown,distance_to_nearest_mrt
0,0,ANG MO KIO AVE 10,1.367602,103.856844,"(1.3676017, 103.8568442, 0.0)",8.383132,0.851339
1,1,ANG MO KIO AVE 4,1.382319,103.838706,"(1.3823185, 103.8387061, 0.0)",9.236619,0.696968
2,2,ANG MO KIO AVE 5,1.376825,103.847556,"(1.3768247, 103.8475557, 0.0)",8.911143,0.617914
3,3,ANG MO KIO AVE 1,1.364355,103.843642,"(1.3643546, 103.8436418, 0.0)",7.465576,0.901853
4,4,ANG MO KIO AVE 3,1.369691,103.840906,"(1.3696908, 103.8409057, 0.0)",7.937842,0.962769
...,...,...,...,...,...,...,...
566,564,JALAN MEMBINA BARAT,1.283347,103.826121,"(1.283347, 103.826121, 0.0)",1.967815,0.330293
567,565,BUANGKOK STH FARMWAY 1,1.388388,103.987492,"(1.38838781397, 103.987491735, 0.0)",20.854348,3.437164
568,566,ALEXANDRA RD,1.273529,103.802049,"(1.2735287, 103.8020495, 0.0)",3.701473,0.165734
569,567,SEMBAWANG RD,1.453349,103.831769,"(1.453349, 103.8317688, 0.0)",16.931287,1.156432


In [291]:
df_lat_long.sort_values('distance_to_downtown')
df_init = pd.merge(df_init, df_lat_long, on='street_name', how='left')
df_init

,Unnamed: 0,street_name,Latitude,Longitude,lat_long,distance_to_downtown,distance_to_nearest_mrt
74,74,DELTA AVE,1.292008,103.828149,"(1.2920076, 103.8281489, 0.0)",1.190634,0.655045
70,70,HENDERSON CRES,1.289199,103.822089,"(1.289199, 103.8220891, 0.0)",1.257930,0.587684
250,248,CLARENCE LANE,1.291909,103.814028,"(1.2919091, 103.8140283, 0.0)",1.279654,0.404853
556,554,MARGARET DRIVE,1.296481,103.810889,"(1.2964806, 103.8108893, 0.0)",1.280812,0.584355
476,474,TIONG BAHRU RD,1.288663,103.819892,"(1.2886625, 103.8198921, 0.0)",1.330928,0.355998
...,...,...,...,...,...,...,...
28,28,JALAN DAMAI,-5.406307,122.626192,"(-5.406307, 122.626192, 0.0)",2218.407215,2203.089885
486,484,JALAN DUSUN,-5.591120,122.731362,"(-5.5911199, 122.7313624, 0.0)",2236.131985,2220.901527
385,383,SMITH ST,37.706114,-97.401073,"(37.706114, -97.401073, 0.0)",15169.926010,15154.309484
405,403,CAMBRIDGE RD,37.592326,-97.526124,"(37.592326, -97.526124, 0.0)",15174.919402,15159.291043


#### Process other venues info

In [294]:
list(df_venues['Venue Category'].unique())

['Theater',
 'Restaurant',
 'Clothing Store',
 'Trail',
 'Playground',
 'Furniture / Home Store',
 'Accessories Store',
 'Supermarket',
 'Bookstore',
 'Outlet Store',
 'Bakery',
 'Shoe Store',
 'Shopping Mall',
 'Bubble Tea Shop',
 'Frozen Yogurt Shop',
 'Sandwich Place',
 'Burger Joint',
 'Coffee Shop',
 'Sporting Goods Shop',
 'Café',
 'Salad Place',
 'Juice Bar',
 'Pharmacy',
 'Food Court',
 'Gym',
 'Market',
 'Noodle House',
 'Stadium',
 'Train Station',
 'Food Truck',
 'Park',
 'Spa',
 'Bus',
 'Recreation Center',
 'Athletics & Sports',
 'Steakhouse',
 'Baseball Stadium',
 'Skating Rink',
 'Karaoke Bar',
 'Museum',
 'Planetarium',
 'Bistro',
 'Hotel',
 'Basketball Court',
 'Ski Area',
 'Soccer Field',
 'Buffet',
 'Health & Beauty Service',
 'Pool',
 'Dessert Shop',
 'Tea Room',
 'Performing Arts Venue',
 'Bowling Alley',
 'Ice Cream Shop',
 'Tennis Court',
 'Electronics Store',
 'Residential Building (Apartment / Condo)',
 'Martial Arts Dojo',
 'Food & Drink Shop',
 'BBQ Joint',
 

In [312]:
df_venues['Venue Category'] = df_venues['Venue Category'].replace(
    ['Bus Station', 'Bus Stop', 'Bus Line'], 
    'Bus', regex = True)

df_venues['Venue Category'] = df_venues['Venue Category'].replace(
    ['Supermarket', 'Department Store', 'Convenience Store', 'Grocery Store'], 
    'Supermarket', regex = True)

df_venues['Venue Category'] = df_venues['Venue Category'].replace(
    ['Shopping Mall', 'Shopping Plaza','Cosmetics Shop','Kids Store','Baby Store',], 
    'Shopping Mall', regex = True)

df_venues['Venue Category'] = df_venues['Venue Category'].replace(
    ['Multiplex', 'Theater', 'Movie Theater', 'Indie Theater', 'College Theater'], 
    'Theater', regex = True)


df_venues['Venue Category'] =df_venues['Venue Category'].replace(
    ['High School','General College & University'], 
    'School', regex = True)

In [313]:
# Create list of selected Venue Categories
venue_list = ['Bus', 'Supermarket', 'Shopping Mall',
              'Theater','Food Court','School']

# Filter venue dataframe with venue list
df_vcat = df_venues.loc[df_venues['Venue Category'].isin(venue_list),]

In [314]:
# Format Venue Categories Dataframe to tidy format
df_vcat = pd.DataFrame(df_vcat.groupby(['street_name', 'Venue Category'])['Venue'].count())
df_vcat = df_vcat.reset_index()
df_vcat = df_vcat.pivot(index='street_name',columns='Venue Category',values='Venue')
df_vcat = df_vcat.reset_index()
df_vcat.fillna(0, inplace = True)
# df_vcat
print(df_vcat.sort_values('Bus'))
print(df_vcat.sort_values('Supermarket'))
print(df_vcat.sort_values('Shopping Mall'))
print(df_vcat.sort_values('Theater'))
print(df_vcat.sort_values('Food Court'))
print(df_vcat.sort_values('School'))

Venue Category             street_name  Bus  Food Court  School  \
0                157 CORPORATION DRIVE  0.0         1.0     0.0   
286                       MARINE DRIVE  0.0         1.0     0.0   
284                    MACPHERSON LANE  0.0         2.0     0.0   
283                     LOWER DELTA RD  0.0         3.0     0.0   
280                      LORONG AH SOO  0.0         1.0     0.0   
..                                 ...  ...         ...     ...   
403                     TAMPINES ST 23  5.0         0.0     0.0   
20                    ANG MO KIO AVE 9  5.0         2.0     0.0   
157                           ELIAS RD  5.0         0.0     0.0   
81              BUKIT BATOK WEST AVE 4  6.0         0.0     0.0   
147                  CORPORATION DRIVE  6.0         1.0     0.0   

Venue Category  Shopping Mall  Supermarket  Theater  
0                         0.0          1.0      0.0  
286                       1.0          1.0      3.0  
284                       0.0    

In [315]:
df_vcat.sort_values('School')

Venue Category,street_name,Bus,Food Court,School,Shopping Mall,Supermarket,Theater
0,157 CORPORATION DRIVE,0.0,1.0,0.0,0.0,1.0,0.0
351,SEMBAWANG DRIVE,1.0,1.0,0.0,1.0,1.0,0.0
350,SEMBAWANG CL,1.0,1.0,0.0,1.0,1.0,0.0
348,SEGAR RD,0.0,1.0,0.0,0.0,0.0,0.0
347,SAUJANA RD,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...
326,PUNGGOL DRIVE,0.0,1.0,1.0,0.0,2.0,0.0
307,PASIR RIS DRIVE 6,0.0,2.0,1.0,0.0,3.0,0.0
447,WATERLOO ST,0.0,1.0,1.0,3.0,1.0,4.0
312,PASIR RIS ST 41,1.0,2.0,1.0,1.0,4.0,1.0


In [316]:
df_main = df_init.merge(df_vcat, how = 'left', on = 'street_name')
df_main = df_main.fillna(0)
df_main['flat_model'] = df_main['flat_model'].str.replace('2-ROOM', '2-room') \
                                    .str.replace('MULTI GENERATION', 'Multi Generation') \
                                    .str.replace('PREMIUM APARTMENT', 'Premium Apartment') \
                                    .str.replace('Premium Apartment.', 'Premium Apartment') \
                                    .str.replace('Premium Apartment Loft', 'Premium Apartment') \
                                    .str.replace('NEW GENERATION', 'New Generation') \
                                    .str.replace('MODEL A', 'Model A') \
                                    .str.replace('Improved-MAISONETTE', 'Maisonette') \
                                    .str.replace('Model A-Maisonette', 'Maisonette') \
                                    .str.replace('Premium ApartmentLoft', 'Premium Apartment') \
                                    .str.replace('flat_type_MULTI-GENERATION', 'flat_type_MULTI GENERATION') \
                                    .str.replace( 'flat_model_PREMIUM Apartment', 'flat_model_Premium Apartment') \
                                    .str.replace('Model A-MAISONETTE', 'Maisonette') \
                                    .str.replace('APARTMENT', 'Apartment') \
                                    .str.replace('MAISONETTE', 'Maisonette') \
                                    .str.replace('IMPROVED', 'Improved') \
                                    .str.replace('SIMPLIFIED', 'Simplified') \
                                    .str.replace('MODEL A-MAISONETTE', 'Maisonette') \
                                    .str.replace('Improved-Maisonette', 'Maisonette') \
                                    .str.replace('Premium Maisonette', 'Maisonette') \
                                    .str.replace('ADJOINED FLAT', 'Adjoined flat') \
                                    .str.replace('TERRACE', 'Terrace') 
                                    

# 'flat_type_MULTI GENERATION',
# 'flat_type_MULTI-GENERATION',
#  'flat_model_PREMIUM Apartment',
#  'flat_model_Premium Apartment',    

df_onehot = pd.get_dummies(df_main[['town', 'flat_type', 'flat_model', 'storey_range']])
df_final = df_main.join(df_onehot)
df_final.drop(['town','month', 'flat_type','flat_model','storey_range', 'street_name', 
               'resale_price','block','_id','lease_commence_date','Latitude','Longitude','lat_long'], 
              axis = 1, inplace = True)
df_final.head()

<ipython-input-316-6152ce4d7aeb>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_main['flat_model'] = df_main['flat_model'].str.replace('2-ROOM', '2-room') \


,floor_area_sqm,remaining_lease,year,price_sqm,distance_to_nearest_mrt,Bus,Food Court,School,Shopping Mall,Supermarket,...,storey_range_28 TO 30,storey_range_31 TO 33,storey_range_31 TO 35,storey_range_34 TO 36,storey_range_36 TO 40,storey_range_37 TO 39,storey_range_40 TO 42,storey_range_43 TO 45,storey_range_46 TO 48,storey_range_49 TO 51
0,44.0,61,2017,5272.727273,0.851339,0.0,5.0,0.0,0.0,3.0,...,0,0,0,0,0,0,0,0,0,0
1,67.0,60,2017,3731.343284,0.696968,0.0,2.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,67.0,62,2017,3910.447761,0.617914,2.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,68.0,62,2017,3897.058824,0.851339,0.0,5.0,0.0,0.0,3.0,...,0,0,0,0,0,0,0,0,0,0
4,67.0,62,2017,3955.223881,0.617914,2.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [329]:
df_onehot = pd.get_dummies(df_main[['town', 'flat_type', 'flat_model', 'storey_range']])
df_final = df_main.join(df_onehot)
df_final.drop(['town','month', 'flat_type','flat_model','storey_range', 'street_name', 
               'resale_price','block','_id','lease_commence_date','Latitude','Longitude','lat_long'], 
              axis = 1, inplace = True)

#  'storey_range_01 TO 03',
#  'storey_range_01 TO 05',
#  'storey_range_04 TO 06',
#  'storey_range_06 TO 10',
#  'storey_range_07 TO 09',
#  'storey_range_10 TO 12',
#  'storey_range_11 TO 15',
#  'storey_range_13 TO 15',
#  'storey_range_16 TO 18',
#  'storey_range_16 TO 20',
#  'storey_range_19 TO 21',
#  'storey_range_21 TO 25',
#  'storey_range_22 TO 24',
#  'storey_range_25 TO 27',
#  'storey_range_26 TO 30',
#  'storey_range_28 TO 30',
#  'storey_range_31 TO 33',
#  'storey_range_31 TO 35',
#  'storey_range_34 TO 36',
#  'storey_range_36 TO 40',
#  'storey_range_37 TO 39',
#  'storey_range_40 TO 42',
#  'storey_range_43 TO 45',
#  'storey_range_46 TO 48',
#  'storey_range_49 TO 51'
    
df_final['storey < 10'] = df_final['storey_range_01 TO 03']\
                        + df_final['storey_range_01 TO 05']\
                        + df_final['storey_range_04 TO 06']\
                        + df_final['storey_range_06 TO 10']\
                        + df_final['storey_range_07 TO 09']\
                
df_final['storey < 15'] = df_final['storey_range_10 TO 12']\
                        + df_final['storey_range_11 TO 15'] 
                                   
df_final['storey < 20'] = df_final['storey_range_16 TO 18']\
                        + df_final['storey_range_16 TO 20'] 
    
df_final['storey < 30'] = df_final['storey_range_19 TO 21']\
                        + df_final['storey_range_21 TO 25']\
                        + df_final['storey_range_22 TO 24']\
                        + df_final['storey_range_06 TO 10']\
                        + df_final['storey_range_25 TO 27']\
                        + df_final['storey_range_26 TO 30']\
                        + df_final['storey_range_28 TO 30']
                        
df_final['story > 30'] = df_final['storey_range_31 TO 33']\
                        + df_final['storey_range_31 TO 35']\
                        + df_final['storey_range_34 TO 36']\
                        + df_final['storey_range_36 TO 40']\
                        + df_final['storey_range_37 TO 39']\
                        + df_final['storey_range_40 TO 42']\
                        + df_final['storey_range_43 TO 45']\
                        + df_final['storey_range_46 TO 48']\
                        + df_final['storey_range_49 TO 51']
                        
df_final.drop(['storey_range_01 TO 03',
                'storey_range_01 TO 05',
                'storey_range_04 TO 06',
                'storey_range_06 TO 10',
                'storey_range_07 TO 09',
                'storey_range_10 TO 12',
                'storey_range_11 TO 15',
                'storey_range_13 TO 15',
                'storey_range_16 TO 18',
                'storey_range_16 TO 20',
                'storey_range_19 TO 21',
                'storey_range_21 TO 25',
                'storey_range_22 TO 24',
                'storey_range_25 TO 27',
                'storey_range_26 TO 30',
                'storey_range_28 TO 30',
                'storey_range_31 TO 33',
                'storey_range_31 TO 35',
                'storey_range_34 TO 36',
                'storey_range_36 TO 40',
                'storey_range_37 TO 39',
                'storey_range_40 TO 42',
                'storey_range_43 TO 45',
                'storey_range_46 TO 48',
                'storey_range_49 TO 51'], 
                axis = 1, inplace = True)

In [330]:
list(df_final.columns)

['floor_area_sqm',
 'remaining_lease',
 'year',
 'price_sqm',
 'distance_to_nearest_mrt',
 'Bus',
 'Food Court',
 'School',
 'Shopping Mall',
 'Supermarket',
 'Theater',
 'town_ANG MO KIO',
 'town_BEDOK',
 'town_BISHAN',
 'town_BUKIT BATOK',
 'town_BUKIT MERAH',
 'town_BUKIT PANJANG',
 'town_BUKIT TIMAH',
 'town_CENTRAL AREA',
 'town_CHOA CHU KANG',
 'town_CLEMENTI',
 'town_GEYLANG',
 'town_HOUGANG',
 'town_JURONG EAST',
 'town_JURONG WEST',
 'town_KALLANG/WHAMPOA',
 'town_LIM CHU KANG',
 'town_MARINE PARADE',
 'town_PASIR RIS',
 'town_PUNGGOL',
 'town_QUEENSTOWN',
 'town_SEMBAWANG',
 'town_SENGKANG',
 'town_SERANGOON',
 'town_TAMPINES',
 'town_TOA PAYOH',
 'town_WOODLANDS',
 'town_YISHUN',
 'flat_type_1 ROOM',
 'flat_type_2 ROOM',
 'flat_type_3 ROOM',
 'flat_type_4 ROOM',
 'flat_type_5 ROOM',
 'flat_type_EXECUTIVE',
 'flat_type_MULTI GENERATION',
 'flat_type_MULTI-GENERATION',
 'flat_model_2-room',
 'flat_model_Adjoined flat',
 'flat_model_Apartment',
 'flat_model_DBSS',
 'flat_model_

In [331]:
df_final

,floor_area_sqm,remaining_lease,year,price_sqm,distance_to_nearest_mrt,Bus,Food Court,School,Shopping Mall,Supermarket,...,flat_model_Simplified,flat_model_Standard,flat_model_Terrace,flat_model_Type S1,flat_model_Type S2,storey < 10,storey < 15,storey < 20,storey < 30,story > 30
0,44.0,61,2017,5272.727273,0.851339,0.0,5.0,0.0,0.0,3.0,...,0,0,0,0,0,0,1,0,0,0
1,67.0,60,2017,3731.343284,0.696968,0.0,2.0,0.0,0.0,1.0,...,0,0,0,0,0,1,0,0,0,0
2,67.0,62,2017,3910.447761,0.617914,2.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
3,68.0,62,2017,3897.058824,0.851339,0.0,5.0,0.0,0.0,3.0,...,0,0,0,0,0,1,0,0,0,0
4,67.0,62,2017,3955.223881,0.617914,2.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839605,142.0,69,1999,3211.267606,0.637828,1.0,2.0,0.0,2.0,1.0,...,0,0,0,0,0,0,1,0,0,0
839606,142.0,70,1999,2873.239437,0.411571,1.0,3.0,0.0,1.0,3.0,...,0,0,0,0,0,1,0,0,0,0
839607,146.0,70,1999,3212.328767,1.670481,0.0,0.0,0.0,1.0,1.0,...,0,0,0,0,0,1,0,0,0,0
839608,146.0,70,1999,3013.698630,1.254211,0.0,3.0,0.0,0.0,2.0,...,0,0,0,0,0,1,0,0,0,0
